## Name: Phuc Tran
### Date: January 31st, 2023
The task of the DALI 23W Data Challenge is to create three data visualizations that accurately describe most of the data. The frameworks that I used for the challenge were plotly and pandas. Specifically, I was interested in plotly as a data visualization library and pandas to parse my dataset.

At the bottom, I have also created a multivariable linear regression with a deep learning model, using PyTorch as my framework.

In [21]:
# Frameworks
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import torch 
import torch.optim as optim
from torch import nn
from torch.utils.data import random_split
import numpy as np
import matplotlib as plt

# Geocoder Module
from geopy.geocoders import Nominatim

# Python Libraries
import collections
from statistics import mean

In [22]:
# Organizing dataset
fileName = "superstore.csv"
superstoreDF = pd.read_csv(fileName)
superstoreDF

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,1/21/2014,1/23/2014,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9990,9991,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9991,9992,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932
9992,9993,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200


#### Purchases vs. Month Line Graph
For the first visualization, I was mainly interested in visualizing the number of purchases for each category. I believed this was a crucial part of the dataset to understand the trend of our purchases. That way, it would be beneficial to sellers who want to know the best time to market their product and to release them to the public. As a result, we can see that There is a plateau in the number of purchases from February to August then a dramatic increase in the purchases towards the American holidays season.

We can also analyze the sub-categories to visualize a more specific set of data.

In [23]:
# Organizing the order dates and the categories of the purchases on this entire dataset
orderDates = superstoreDF['Order Date']
categories = superstoreDF['Category']
subcategories = superstoreDF['Sub-Category']
region = superstoreDF['Region']

# IMPORTANT: Select what we want to compare to according to the order date
ySubject = subcategories
monthlyCategories = {}

# Iterate through the order dates and categories
for index, (orderDate, category) in enumerate(zip(orderDates, ySubject)):
    month = int(orderDate.split("/")[0])
    # Update the month's category purchases
    if category not in monthlyCategories:
        monthlyCategories[category] = {}
    else:
        monthlyCategories[category][month] = 1 + monthlyCategories[category].get(month, 0)

In [24]:
# The list of categories 
categoryList = list(monthlyCategories.keys())
pdList = []

# Creating a dataframe for each category
for category in categoryList:
    orderedDict = collections.OrderedDict(sorted(monthlyCategories[category].items()))
    color = [category]*len(orderedDict.keys())
    furnitureBarDict = { "Month" : orderedDict.keys(),
                         "Purchases":orderedDict.values(),
                          "Category":color}
    pdList.append(pd.DataFrame.from_dict(furnitureBarDict))

    # Recombining the dataframe
result = pd.concat(pdList)

In [25]:
# Displaying the graph
fig = px.line(result, x="Month", y="Purchases", title="Purchases of Categories in a Span of a Year", color='Category')
fig.show()

#### Sales Compared to a USA Heat Map 
I was particularly interested in creating a heat map because it provided us a lot of opportunities to focus on certain regions. When I compared the number of sales to the region, I saw a disprecancy between each region. With this, we could analyze the standard income in each state. When I saw that the city was given in the dataset, I thought that we could see the number of sales being made within that city. However, in order to make the statement more generalized, I decided to group the cities by states instead. Sellers can use this heatmap to decide at what price range should they sell their products according to each state.

In [26]:
# Create a dictionary that maps the city to the state code
from city_to_state import city_to_state_dict, s_state_to_abbrev

# Parsing for the city
cities = superstoreDF['City']
coordinates = []
cityDictionary = {}

# Mapping the city to the state that it belongs to
geolocator = Nominatim(user_agent="LocatingCoordinates")
for location in cities:
    if location in city_to_state_dict:
        state = city_to_state_dict[location]
        stateAbbrev = s_state_to_abbrev[state]
        cityDictionary[location] = stateAbbrev

In [27]:
# Create a dictionary that maps total sales to each city
sales = superstoreDF['Sales']
salesDictionary = {}

# Summing up the sales
for city, sale in zip(cities, sales):
    if city in city_to_state_dict:
        state = cityDictionary[city]
        if state not in salesDictionary:
            salesDictionary[state] = sale
        else:
            salesDictionary[state] += sale

In [28]:
# Creating the dataframe with each state and the total sales
d = {'State Code': list(salesDictionary.keys()),
     'Sales': list(salesDictionary.values())
}
df = pd.DataFrame(data=d)

In [29]:
# Plotting the heatmap of the supermarket purchases by each state
fig = go.Figure(data=go.Choropleth(
    locations=df['State Code'], # Spatial coordinates
    z = df['Sales'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'Greens',
    colorbar_title = "USD",
))
fig.update_layout(
    title_text = 'Supermarket Purchases by States',
    geo_scope='usa', # limite map scope to USA
)
fig.show()

#### Average Shipping Date compared to Region
I was interested to see if there was a discrepancy between the average shipping date and the region in which the order was made. From the bar graph, it seems that the supermarket is doing well in terms of having a consistent shipping time. West has a slightly higher average shipping date, but it is well within the margins of the other regions.

In [30]:
# Initializing the dataFrame columns
orderDate = superstoreDF['Order Date']
shippingDate = superstoreDF['Ship Date']
regions = superstoreDF['Region']
uniqueRegions = regions.unique()

# Store the dictionary and the shipping time
shippingTimeDict = {key: [] for key in uniqueRegions}
for order, shipping, region in zip(orderDate, shippingDate, regions):
    # Organize the shipping and order time
    monthShip, dateShip, yearShip = shipping.strip().split("/")
    monthOrder, dateOrder, yearOrder = order.strip().split("/")
    shippingTime = (int(yearShip) - int(yearOrder)) * 365 + (int(monthShip) - int(monthOrder)) * 30 + (int(dateShip) - int(dateOrder)) 
    shippingTimeDict[region].append(shippingTime)

# Calculating the average shipping date
for uniqueRegions in shippingTimeDict.keys():
    shippingTimeDict[uniqueRegions] = mean(shippingTimeDict[uniqueRegions])    

In [31]:
import plotly.express as px
# Graphing the average shipping date compared to each region
df = pd.DataFrame(list(shippingTimeDict.items()), columns=['Region','Average Shipping Date'])
fig = px.bar(df, x="Region", y="Average Shipping Date", color = "Region", title="Average Shipping Date of Each Region")
fig.show()

#### Multivariable Linear Regression with Deep Learning Model
For the Creative Challenge, I wanted to showcase a deep learning model that could predict the profit based on the given data. Most importantly, I believed that the sales, quantities, and discounts would have the highest influence on the profit shown. I also thought that these data would be relatively easier to model compared something such as Order ID. I created the model with a linear multiplication functions and non-linear functions. I wanted to include the ReLU() function to perform some of the nonlinearity these correlations might have.

With the result, I saw that the data did not have as much correlation with the profit as I would have wanted. There is a possible correlation between the profits and the discounts. My results are inconclusive, but I would love to further explore other data that the superstore might have that can help us indicate the profit such as the general comments and ratings of the order. From the result, tt seems that the profit is a too wide of an output range to be effectively predicted with a multivariable linear regression. For the next iteration, I would classify the profits to be within ranges of prices and predict the items within such ranges. 

In [40]:
# Preprocessing data
# Set up inputs as sales, quantities, and discounts
# Set output as profit
sales = superstoreDF['Sales']
quantities = superstoreDF['Quantity']
discounts =  superstoreDF['Discount']
profit = superstoreDF['Profit']
# Concatenating the input arrays
inputDF = pd.concat([sales, pd.concat([quantities, discounts], axis=1)], axis=1)
inputArray = torch.tensor(inputDF.values, dtype=torch.float32)
numberofLabels = inputArray.size(dim=1)
# Creating output array
y_truth = torch.tensor(profit, dtype=torch.float32).unsqueeze(0).T
# Combining dataset and splitting them to training and validation sets
dataset = torch.concat((y_truth, inputArray), 1)

In [43]:
### Model definition ###
# Performing Multivariable Linear Regression
# Defining parameter W and the biases
W = torch.randn((1, numberofLabels), requires_grad=True, dtype=torch.float32)
b = torch.randn(1, requires_grad=True, dtype=torch.float32)

# Then we define the prediction model
def model(x_input):
    hidden_units = [128, 64, 32, 16, 8, 1]
    current_input = x_input
    outputs = [x_input]
    for index, hidden in enumerate(hidden_units):
        # Declaring the dimension of inputs and outputs
        if index == 0:
            inputUnit = numberofLabels
            currentInput = x_input
        else:
            inputUnit = hidden_units[index - 1]  
            currentInput = outputs[-1]
            
        # One linear layer, one batch normalization, then one ReLU layer
        outputUnit = hidden_units[index]
        linear = nn.Linear(inputUnit, outputUnit)
        linear_out = linear(currentInput)
        nonlinear = nn.ReLU()
        outputs.append(nonlinear(linear_out))
    return outputs[-1]


### Loss function definition ###
def loss(y_predicted, y_target):
    return ((y_predicted - y_target)**2).sum()

In [44]:
### Training the model ###

# Setup the optimizer object, so it optimizes W and b.
optimizer = optim.SGD([W, b], lr=0.1, weight_decay = 0.1, momentum = 0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

MSE = []
# Learning with 2000 epochs
for t in range(20):
    # Setting the gradient to 0
    optimizer.zero_grad()
    # Compute the current predicted y's from x_dataset
    y_predicted = model(inputArray)
    # Calculate loss and gradient of loss
    current_loss = loss(y_predicted, y_truth)
    current_loss.backward()
    MSE.append(current_loss.item())
    # Update weights and biases accordingly.
    optimizer.step()
    scheduler.step()
    print("epoch", str(t), "loss", str(current_loss.item()))


epoch 0 loss 556601088.0
epoch 1 loss 556601088.0
epoch 2 loss 556601088.0
epoch 3 loss 551799296.0
epoch 4 loss 555356096.0
epoch 5 loss 541832064.0
epoch 6 loss 556474560.0
epoch 7 loss 554327808.0
epoch 8 loss 556601088.0
epoch 9 loss 556600576.0
epoch 10 loss 553613760.0
epoch 11 loss 554608192.0
epoch 12 loss 554865472.0
epoch 13 loss 554931840.0
epoch 14 loss 556523776.0
epoch 15 loss 556601088.0
epoch 16 loss 555639296.0
epoch 17 loss 556580096.0
epoch 18 loss 556601088.0
epoch 19 loss 552715968.0


In [45]:
# Setting up visualization of weights with the input features
weightList = []
inputs = ["Sales", "Quantities", "Discounts"]
for index, weight in enumerate(torch.detach(W.T).tolist()):
    weightList.append(float(weight[0]))
weights_df = pd.DataFrame(dict(
    inputs = inputs,
    weights = weightList
))

# Visualizing the error
epochs = []
for epoch in range(len(MSE)):
    epochs.append(epoch)
df = pd.DataFrame(dict(
    epoch = epochs,
    error = MSE
))
fig = px.line(df, x="epoch", y="error", title="Mean Squared Error of Multivariable Model") 
fig.show()

In [36]:
weights_df

,inputs,weights
0,Sales,0.389197
1,Quantities,1.243281
2,Discounts,-0.265049


In [37]:
print(y_truth)
print(y_predicted)

tensor([[ 41.9136],
        [219.5820],
        [  6.8714],
        ...,
        [ 19.3932],
        [ 13.3200],
        [ 72.9480]])
tensor([[0.],
        [0.],
        [0.],
        ...,
        [0.],
        [0.],
        [0.]], grad_fn=<ReluBackward0>)
